In [49]:
from __future__ import print_function
import aerospike
from aerospike import predicates
import logging
import time

logging.getLogger().setLevel(logging.WARNING)

maxLoops = 100

aeroHost = ("127.0.0.1", 3000)
aeroSet = "customers"

# Поскольку предупреждали, что запуск будет производиться на чистом аэроспайке,
#   мы не знаем, какой там неймспейс, в итоге берем первый попавшийся :)
aeroNameSpace = client.info_node("namespaces", aeroHost, None).split("\n")[0].split("\t")[1]
print("Cluster's current namespace is",aeroNameSpace)

Cluster's current namespace is test


In [39]:
# Configure the client
config = {
    'hosts': [ aeroHost ]
}

In [40]:
# Create a client and connect it to the cluster
try:
    client = aerospike.client(config).connect()
except:
    import sys
    print("failed to connect to the cluster with", config['hosts'])
    sys.exit(1)

In [63]:
client.index_string_create(aeroNameSpace, aeroSet, "phone_number", aerospike.INDEX_STRING, aeroNameSpace+_+aeroSet+"_phone_number_idx")
aeroQuery = client.query(aeroNameSpace, aeroSet)

ParamError: (-2, 'policy must be a dict', 'src/main/policy.c', 557, False)

In [54]:
aeroQuery.select("lifetime_value")
aeroQuery.where(predicates.equals("phone_number", 2))
results = aeroQuery.results( {'total_timeout':2000})
print(results)

IndexNotFound: (201, 'AEROSPIKE_ERR_INDEX_NOT_FOUND', 'src/main/aerospike/aerospike_query.c', 345, False)

In [56]:
def add_customer(customer_id, phone_number, lifetime_value):
    try:
        client.put( (aeroNameSpace, aeroSet, customer_id), {
            'phone_number': phone_number,
            'lifetime_value': lifetime_value
         })
    except Exception as e:
        logger.error("error: {0}".format(e))

In [57]:
def get_ltv_by_id(customer_id):
    try:
        (key, metadata, record) = client.get((aeroNameSpace, aeroSet, customer_id))
        return record.get("lifetime_value")
    except Exception as e:
        logger.error('Requested non-existent customer ' + str(customer_id))

In [43]:
def get_ltv_by_phone(phone_number):
    # Здесь, вероятно, потребуется как-то искать по доп. индексу по телефону
    # Либо сделать ход конем, что, по-моему, даже интереснее,
    #   и создать отдельный сет (таблицу) для связи телефон--кастомер_ид, где ключом выступает уже телефон
    #   или для этих целей в аэроспайке есть какие-то проекции... :)
    try:
        (key, metadata, record) = client.get(customer_id)
        return record.get('ltv')
    except Exception as e:
        logging.error('Requested phone number is not found ' + str(phone_number))

In [60]:
startTime = time.time()
for i in range(0, maxLoops):
    add_customer(i,i,i + 1)
print("Insert time by id: ", (time.time()-startTime))

Insert time by id:  0.03892064094543457


In [61]:
print ("All is OK! :)")

All is OK! :)


In [29]:
# Close the connection to the Aerospike cluster
client.close()